<a href="https://colab.research.google.com/github/mattalejo/thesis/blob/time2vec/thesis_1day.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
!git clone --branch time2vec https://github.com/mattalejo/thesis
sys.path.append("/content/thesis")

Cloning into 'thesis'...
remote: Enumerating objects: 949, done.
remote: Counting objects: 100% (949/949), done.
remote: Compressing objects: 100% (512/512), done.
remote: Total 949 (delta 251), reused 829 (delta 187), pack-reused 0
Receiving objects: 100% (949/949), 31.80 MiB | 37.00 MiB/s, done.
Resolving deltas: 100% (251/251), done.


# Import libraries

In [2]:
import yfinance as yf
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
import math as m

from sklearn.preprocessing import StandardScaler
# from tensorboard_logger import configure

import copy
import sys
import os

import prep_data
import train

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader, TensorDataset

from models.transformer import Transformer
from models.rnn import RNN
from models.autoformer import Autoformer

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [3]:
if torch.cuda.is_available():
 dev = "cuda:0"
else:
 dev = "cpu"
device = torch.device(dev)

print(dev)

cpu


# Load data from `yfinance`

In [ ]:
%%time
model_transformer, loss_transformer, train_pred_transformer, test_pred_transformer, scaler = train.train(
                                                                                        model=Transformer(dropout=0.4, embed_mode="pe"),
                                                                                        seq_len=64,
                                                                                        horizon=1,
                                                                                        max_epoch=10,
                                                                                        batch_size=32,
                                                                                        loss=nn.MSELoss(),
                                                                                        optimizer=optim.Adam,
                                                                                        lr=1e-4,
                                                                                        device=dev
                                                                                    )

[*********************100%%**********************]  1 of 1 completed


                   Open         High          Low        Close    Adj Close  \
Date                                                                          
1990-01-02  1120.760010  1120.760010  1120.760010  1120.760010  1120.471924   
1990-01-03  1114.739990  1114.739990  1114.739990  1114.739990  1114.453369   
1990-01-04  1129.250000  1129.250000  1129.250000  1129.250000  1128.959717   
1990-01-05  1128.609985  1128.609985  1128.609985  1128.609985  1128.319824   
1990-01-08  1131.369995  1131.369995  1131.369995  1131.369995  1131.079102   

            Volume  
Date                
1990-01-02       0  
1990-01-03       0  
1990-01-04       0  
1990-01-05       0  
1990-01-08       0  


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [5]:
%%time
model_rnn, loss_rnn, train_pred_rnn, test_pred_rnn, scaler = train.train(
                                                        model=RNN(dropout=0.2),
                                                        seq_len=64,
                                                        horizon=1,
                                                        max_epoch=10,
                                                        batch_size=32,
                                                        loss=nn.MSELoss(),
                                                        optimizer=optim.Adam,
                                                        lr=1e-4,
                                                        device=dev
                                                    )

[*********************100%%**********************]  1 of 1 completed


                   Open         High          Low        Close    Adj Close  \
Date                                                                          
1990-01-02  1120.760010  1120.760010  1120.760010  1120.760010  1120.471924   
1990-01-03  1114.739990  1114.739990  1114.739990  1114.739990  1114.453369   
1990-01-04  1129.250000  1129.250000  1129.250000  1129.250000  1128.959717   
1990-01-05  1128.609985  1128.609985  1128.609985  1128.609985  1128.319824   
1990-01-08  1131.369995  1131.369995  1131.369995  1131.369995  1131.079102   

            Volume  
Date                
1990-01-02       0  
1990-01-03       0  
1990-01-04       0  
1990-01-05       0  
1990-01-08       0  
Epoch 0 | train loss 0.536345642391998 | test_loss 1.2040023803710938 | wall_time 12.686029434204102 | process_time 8.721447042
Epoch 1 | train loss 0.5352667629128636 | test_loss 1.20502769947052 | wall_time 6.4207987785339355 | process_time 6.356306580000002
Epoch 2 | train loss 0.534788566060585